In [1]:
import itertools
from copy import deepcopy

import numpy as np

# from value_iteration import run_reinforcement_learning

In [179]:
lam = 2
mu1 = 0.1
mu2 = 2
K = 25

In [180]:
V = np.zeros(shape=(K+1, 2, 2))

In [ ]:
def create_state_space_for_two_servers(queue_capacity):
    """
    Create a state space for a given queue capacity.
    """
    return tuple(itertools.product(range(queue_capacity + 1), range(2), range(2)))

In [ ]:
def get_all_actions(Q, all_states, queue_capacity):
    """
    Get all actions for a given state.
    """
    actions = {}
    for state in all_states:
        actions[state] = exploit_policy(state, Q)
    return actions

In [181]:
def explore_policy(state):
    return [0.34, 0.33, 0.33]

In [182]:
def exploit_policy(state, Q):
    return np.argmax(Q[state[0], state[1], state[2], :])

In [183]:
def select_action(state, epsilon, actions, Q_table):
    if np.random.rand() < epsilon: # policy == 'explore'
        return np.random.choice(actions)
    else: # Exploit
        return exploit_policy(state, Q_table) 

In [239]:
def state_transition(state, par):
    tot_prob = sum(par[:-1])
    lam, mu1, mu2, K = par
    # tot_prob = lam + mu1 * state[1] + mu2 * state[2]

    # probs = [
    #     lam / tot_prob, (mu1 * state[1]) / tot_prob, (mu2 * state[2]) / tot_prob
    # ]

    prob_arr = lam / tot_prob
    prob_leave1 = mu1 / tot_prob
    prob_leave2 = mu2 / tot_prob
    
    # actions_and_probs = tuple(enumerate(probs))
    next_transition = np.random.choice(
        [0, 1, 2], p=[prob_arr, prob_leave1, prob_leave2]
    )
    
    if next_transition == 0 and state[0] < K:
        state[0] += 1
    elif next_transition == 1 and state[1] == 1:
        state[1] -= 1
    elif next_transition == 2 and state[2] == 1:
        state[2] -= 1
    return state

In [240]:
def state_transition_monkey(state, action): # return state, cost/reward
    s0, s1, s2 = state
    if s0 == 0:
        if action == 0:
            return [s0, s1, s2], -(s0 + s1 + s2) ** 2
        else:
            return [s0, s1, s2], -1000
    else:
        if s1 == 1 and action == 1:
            return [s0, s1, s2], -1000
        elif s1 == 0 and action == 1:
            return [s0-1, s1+1, s2], -(s0 + s1 + s2) ** 2
        elif s2 == 1 and action == 2:
            return [s0, s1, s2], -1000
        elif s2 == 0 and action == 2:
            return [s0-1, s1, s2+1], -(s0 + s1 + s2) ** 2
        elif action == 0:
            return [s0, s1, s2], -(s0 + s1 + s2) ** 2
    raise ValueError(f"check {action, [s0, s1, s2]}") 

In [241]:
def q_learning(par, gamma=0.99, alpha=0.2, n_episodes=100, n_steps=1000):
    
    states = np.zeros(shape=(K+1, 2, 2))
    actions = [0, 1, 2]

    Q = np.ones(shape=(K+1, 2, 2, len(actions))) * (-K)
    
    while True:
        state = [0, 0, 0]    
        old_Q = deepcopy(Q)

        for step in range(n_steps):
            # Observe event
            old_state = deepcopy(state)
            state = state_transition(state, par)

            # Choose action
            action = select_action(state, 0.5, actions, Q)

            # Apply action / observe next state and rewards
            new_state, reward = state_transition_monkey(state, action)

            # Compute delta_t
            delta = np.abs(np.array(new_state) - np.array(old_state)).max()

            # Update Q value
            best_achievable_pol = exploit_policy(new_state, Q)
            best_achievable_rew = Q[new_state[0], new_state[1], new_state[2], best_achievable_pol]

            
            Q[state[0], state[1], state[2], action] += alpha * (
                reward + gamma * best_achievable_rew - Q[state[0], state[1], state[2], action]
            )
        
        delta_Q = np.abs(old_Q - Q)
        if delta_Q.max() < 1e-1:
            break
        
    return Q

In [242]:
lam = 2
mu1 = 0.1
mu2 = 2
K = 40

In [243]:
par = [lam, mu1, mu2, K]
Q = q_learning(par=par, alpha=0.01, gamma=0.99)

In [232]:
def create_state_space_for_two_servers(queue_capacity):
    """
    Create a state space for a given queue capacity.
    """
    return tuple(itertools.product(range(queue_capacity + 1), range(2), range(2)))

In [233]:
def get_all_actions(Q, all_states, queue_capacity):
    """
    Get all actions for a given state.
    """
    actions = {}
    for state in all_states:
        actions[state] = exploit_policy(state, Q)
    return actions

In [234]:
state_space = create_state_space_for_two_servers(K)
actions = get_all_actions(Q, state_space, K)

In [235]:
tuple((key, value) for key, value in actions.items())

(((0, 0, 0), 0),
 ((0, 0, 1), 0),
 ((0, 1, 0), 0),
 ((0, 1, 1), 0),
 ((1, 0, 0), 2),
 ((1, 0, 1), 0),
 ((1, 1, 0), 0),
 ((1, 1, 1), 0),
 ((2, 0, 0), 2),
 ((2, 0, 1), 0),
 ((2, 1, 0), 0),
 ((2, 1, 1), 0),
 ((3, 0, 0), 2),
 ((3, 0, 1), 0),
 ((3, 1, 0), 0),
 ((3, 1, 1), 0),
 ((4, 0, 0), 2),
 ((4, 0, 1), 0),
 ((4, 1, 0), 0),
 ((4, 1, 1), 0),
 ((5, 0, 0), 2),
 ((5, 0, 1), 0),
 ((5, 1, 0), 0),
 ((5, 1, 1), 0),
 ((6, 0, 0), 2),
 ((6, 0, 1), 0),
 ((6, 1, 0), 0),
 ((6, 1, 1), 0),
 ((7, 0, 0), 2),
 ((7, 0, 1), 0),
 ((7, 1, 0), 0),
 ((7, 1, 1), 0),
 ((8, 0, 0), 2),
 ((8, 0, 1), 0),
 ((8, 1, 0), 0),
 ((8, 1, 1), 0),
 ((9, 0, 0), 1),
 ((9, 0, 1), 0),
 ((9, 1, 0), 0),
 ((9, 1, 1), 0),
 ((10, 0, 0), 2),
 ((10, 0, 1), 0),
 ((10, 1, 0), 0),
 ((10, 1, 1), 0),
 ((11, 0, 0), 2),
 ((11, 0, 1), 0),
 ((11, 1, 0), 0),
 ((11, 1, 1), 0),
 ((12, 0, 0), 2),
 ((12, 0, 1), 0),
 ((12, 1, 0), 0),
 ((12, 1, 1), 0),
 ((13, 0, 0), 2),
 ((13, 0, 1), 0),
 ((13, 1, 0), 0),
 ((13, 1, 1), 0),
 ((14, 0, 0), 2),
 ((14, 0, 1), 